<a href="https://colab.research.google.com/github/sudhang/css-nlp/blob/master/ngram/N_Gram_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will use train an n-gram MLE-based model.  We will rely on the library [nltk](https://www.nltk.org/api/nltk.lm.html) for this purpose.



### Installations

In [ ]:
# Nothing

## Imports

To use the llama2 models from huggingface, we need to input an access token.

In [ ]:
GDRIVEPATH = "/content/drive/MyDrive/TU/Sem 4/NLP"

In [ ]:
import pandas as pd
import pandas as pd
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Dataset

In [ ]:
df = pd.read_csv(f"{GDRIVEPATH}/data/nyt_train.csv")

df = df[["content"]]
display(df)

,content
0,"WASHINGTON — Three years ago, President Barack..."
1,When Honda Motor Company said two months ago t...
2,WASHINGTON — Four months after a historic acco...
3,WASHINGTON — Within hours of opening an invest...
4,"WASHINGTON — The F.B.I. director, James B. Com..."
...,...
8580,Senator Marco Rubio of Florida won the Puerto ...
8581,"EXETER, N.H. — Eight years after aggressively ..."
8582,"PRINCETON, Ore. — The armed occupation of a wi..."
8583,"WASHINGTON — As North Korea’s reclusive ruler,..."


## PARAMS

In [ ]:
N = 6

## Preprocessing

In [ ]:
the_corpus = list(df['content'].apply(nltk.word_tokenize))
train_data, padded_sents = padded_everygram_pipeline(N, the_corpus)

## Training

In [ ]:
the_model = nltk.lm.MLE(N)
the_model.fit(train_data, padded_sents)

KeyboardInterrupt: ignored

In [ ]:
len(the_model.vocab)

In [ ]:
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, prompt=["Graz"], random_seed=None):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed, text_seed=prompt):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

prompt = "MEXICO CITY — Mexico’s most prominent human rights lawyers, journalists and anti-corruption activists have been targeted by advanced spyware sold to the Mexican government on the condition that it be used only to investigate criminals and terrorists. The targets include lawyers looking into the mass disappearance of 43 students, a highly respected academic who helped write anti-corruption legislation, two of Mexico’s most influential journalists and an American representing victims of sexual abuse by the police."
prompt_tokens = nltk.word_tokenize(prompt)
the_gen_text = prompt + generate_sent(the_model, num_words=2000, prompt=prompt_tokens)
the_gen_text

'MEXICO CITY — Mexico’s most prominent human rights lawyers, journalists and anti-corruption activists have been targeted by advanced spyware sold to the Mexican government on the condition that it be used only to investigate criminals and terrorists. The targets include lawyers looking into the mass disappearance of 43 students, a highly respected academic who helped write anti-corruption legislation, two of Mexico’s most influential journalists and an American representing victims of sexual abuse by the police.The police have arrested several men for sexually abusing and exploiting women and girls, according to the United Nations, François Delattre, singled out “ strong political instincts ” as Ms. Haley ’ s trademark . Mr. Delattre recalled how Ms. Haley had orchestrated a White House lunch for the Security Council diplomats . “ Now, does everybody like Nikki? ” the president asked . “ They weren ’ t outgoing, ” she said . “ We do not want to stand in the way of a simple, fair tax s

## Save The Model

To save the adapter, we rely on the Huggingface hub

In [ ]:
import pickle

with open(f"{GDRIVEPATH}/models/ngram_nyt_{N}.pkl", 'wb') as f:
    pickle.dump(the_model, f)